In [82]:
Ruedas = {
    "base": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "I"   : "EKMFLGDQVZNTOWYHXUSPAIBRCJ",
    "II"  : "AJDKSIRUXBLHWTMCQGZNPYFVOE",
    "III" : "BDFHJLCPRTXVZNYEIWGAKMUSQO",
    "IV"  : "ESOVPZJAYQUIRHXLNFTGKDCMWB",
    "V"   : "VZBRGITYUPSDNHLXAWMJQOFECK",
    "VI"  : "JPGVOUMFYQBENHZRDKASXLICTW",
    "VII" : "NZJHGRCXMYSWBOUFAIVLPEKQDT",
    "VIII": "FKQHTLXOCBJSPDZRAMEWNIUYGV"
}

Ruedas_griegas = {
    "base" : "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "beta" : "LEYJVCNIXWPBQMDRTAKZGFUHOS",
    "gamma": "FSOKANUERHMBTIYCWLQPZXVGJD"
}

Reflectores = {
    "base": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "A"   : "EJMZALYXVBWFCRQUONTSPIKHGD",
    "B"   : "YRUHQSLDPXNGOKMIEBFZCWVJAT",
    "C"   : "FVPJIAOYEDRZXWGCTKUQSBNMHL"
}

In [87]:
class Rotor:
    def __init__(self, rueda: str):      
        self.wiring = rueda

    def siguiente(self, indice):
        letra_a_buscar = self.wiring[self.indice]
        indice_siguiente = Ruedas["base"].find(letra_a_buscar)
        return indice_siguiente

    def revertir(self, indice):
        letra_en_base = Ruedas["base"][indice]
        indice_de_salida = self.wiring.find(letra_en_base)    
        return indice_de_salida

In [88]:
class Reflector:
    def __init__(self, reflector: str):
        self.wiring = reflector
    
    def reflejar(self, indice):
        """
        `Indice` representa la letra de base[indice].
        """
        letra_a_buscar = self.wiring[self.indice]
        indice_siguiente = Reflectores["base"].find(letra_a_buscar)
        return indice_siguiente

In [86]:
class Plugboard:
    def __init__(self, parametros):
        """
        El parámetro `mapping` es una lista de duplas, de la forma
        ```
        [(letra1, letra2), (letra3, letra4)]
        ```
        
        De forma que se construirá un mapa tal que letra1 -> letra2, etc...
        """
        
        self.base = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.mapping = {}

        # Nos guardamos primero un mapa de la forma A -> A, B -> B, ...
        for letra in self.base:
            self.mapping[m] = m

        # Y ahora lo sustituimos por lo que nos han mandado
        for dupla in parametros:
            self.mapping[dupla[0]] = dupla[1]
            self.mapping[dupla[1]] = dupla[0]
    
    def siguiente(self, letra: str):
        return self.mapping[letra]

In [79]:
class Enigma:
    def __init__(self):
        print()

In [84]:
rotor = Rotor(Ruedas["I"])


In [85]:
rotor.siguiente()

4